In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Create webdriver instance 

In [2]:
summary_driver = webdriver.Firefox()
table_driver = webdriver.Firefox()

summary_driver.implicitly_wait(10)
table_driver.implicitly_wait(10)

## Seed proteins

In [ ]:
seed_protein = '35ML'

## Process for one seed protein

In [ ]:
summary_driver.get("https://www.ncbi.nlm.nih.gov/Structure/pdb/" + seed_protein)

In [ ]:
# Find protein length
rulers = summary_driver.find_elements_by_class_name('mol-svg-ruler-text')
query_protein_length = int(rulers[-1].get_attribute('innerHTML'))

In [ ]:
a_links = summary_driver.find_elements_by_tag_name('a')

In [ ]:
# The sequence/domain searches have URLs of this form. The first one is always the gray-box, or the entire sequence,
# and not a single domain

base_url = '{\'animVal\': \'https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=' 

In [ ]:
url_to_search = ''

for link in a_links:
    href = link.get_attribute('href')
    if type(href) == dict:
        url_to_search = href['baseVal']
        break

# Time to query the table

In [ ]:
table_driver.get(url_to_search)

In [ ]:
# Get the protein query string
query_protein_string = table_driver.find_element_by_id('query-id').get_attribute('data-querychain').replace('_', '')

In [ ]:
# Change filter options at top
redundancy_select = Select(table_driver.find_element_by_id('subset-select'))
display_select = Select(table_driver.find_element_by_id('display-selection'))

redundancy_select.select_by_visible_text('Low redundancy')
display_select.select_by_visible_text('Table')

In [ ]:
# Refresh table with new options
refresh_table_button = table_driver.find_element_by_id('seqlist-button')
refresh_table_button.click()

In [ ]:
# For navigating whole length of table
pages = []
try:
    num_pages_menu = Select(table_driver.find_element_by_id('page-select-1'))
    for o in num_pages_menu.options:
        pages.append(o.txt)
except:
    print("All data on a single page")

In [ ]:
# Extract the data from a page of VAST
table = table_driver.find_element_by_id('vast-table')

In [ ]:
table_html = table.get_attribute('outerHTML')
table_bsoup = BeautifulSoup(table_html, 'html.parser')
rows = table_bsoup.find_all('tr')
rows.pop(0) # First row is always the table header

In [ ]:
names = []
descriptions = []
rmsds = []
aligned_residues = [] 
seq_ids = []
seed_proteins = []
urls = []

for r in rows:
    row_values = r.text.strip().split('\n')
    # Filter conditions 
    # 1. Only 1 sequence of PDB entry (no domains)
    # 2. Sequence identity < 90%
    # 3. Matched residue length is at least 50% of the length of query protein
    if len(row_values[0]) == 5 and float(row_values[5]) < 90 and int(row_values[1]) * 2 > query_protein_length:
        names.append(row_values[0])
        descriptions.append(row_values[6])
        rmsds.append(row_values[4])
        aligned_residues.append(row_values[1])
        seq_ids.append(row_values[5])
        seed_proteins.append(seed_protein)
        
        # This is the link to the same table, queried for the matched sequence (not just PDB ID)
        urls.append('https://www.ncbi.nlm.nih.gov/Structure/vast/' + r.find_all('a')[1]['href'])

In [ ]:
protein_dict = {
    'name': names,
    'seed_protein': seed_proteins,
    'description': descriptions,
    'rmsd': rmsds,
    'aligned_residue': aligned_residues,
    'sequence_id': seq_ids,
    'url': urls
}

df = pd.DataFrame(protein_dict)

In [ ]:
df

# Test method for finding sequence within a PDB entry in summary page

In [ ]:
data_table = summary_driver.find_element_by_class_name('bucontent')
data_table_bsoup = BeautifulSoup(data_table.get_attribute('outerHTML'), 'html.parser')

In [ ]:
molecule_labels = data_table_bsoup.find_all('td', {'class': 'molecule-label-cell'})
all_row_labels = []
for row in molecule_labels:
    row_list = []
    icons_in_row = row.find_all('div', {'class': 'protein-table-chain-label'})
    for i in icons_in_row:
        row_list.append(i.text)
    all_row_labels.append(row_list)

In [ ]:
all_row_labels

Find the links in the rulers

In [ ]:
ruler_cells = data_table_bsoup.find_all('div', {'class': 'annot-img-block floatClear'})

In [ ]:
all_row_links = []
for row in ruler_cells: # Convenient that the first g will always be the grey ruler box
    g = row.find('g')
    grey_ruler_link = g.find('a')
    all_row_links.append(grey_ruler_link['xlink:href'])
    

In [ ]:
all_row_links

### Below function gets the proper VAST url for a protein chain

In [3]:
def get_seed_protein_url(pdb_id: str):
    summary_driver.get("https://www.ncbi.nlm.nih.gov/Structure/pdb/" + pdb_id[:4])
    
    data_table = summary_driver.find_element_by_class_name('bucontent')
    data_table_bsoup = BeautifulSoup(data_table.get_attribute('outerHTML'), 'html.parser')
    
    ruler_cells = data_table_bsoup.find_all('div', {'class': 'annot-img-block floatClear'})
    all_row_links = []
    for row in ruler_cells: # Convenient that the first g will always be the grey ruler box
        g = row.find('g')
        grey_ruler_link = g.find('a')
        all_row_links.append(grey_ruler_link['xlink:href'])
    
    # Conditionals for if we are given chain ids or not
    if len(pdb_id) == 4:
        return all_row_links[0] # return first and only grey ruler link
    elif len(pdb_id) == 5:
        # Find which row should be returned - we match the chain id to the row
        molecule_labels = data_table_bsoup.find_all('td', {'class': 'molecule-label-cell'})
        all_row_labels = []
        for row in molecule_labels:
            row_list = []
            icons_in_row = row.find_all('div', {'class': 'protein-table-chain-label'})
            for i in icons_in_row:
                row_list.append(i.text)
            all_row_labels.append(row_list)
            
        for index, label_list in enumerate(all_row_labels):
            if pdb_id[4] in label_list:
                return all_row_links[index]
        
        return("ERROR - unable to find chain ID")
    else:
        return("ERROR - invalid PDB ID length")

# Time to put it all together into a single loopable process

In [4]:
def get_info_from_url(url: str, known_urls: list, unknown_urls: list, aligned_threshold=0.5):
    table_driver.get(url)
    
    names = []
    descriptions = []
    rmsds = []
    aligned_residues = [] 
    seed_protein_total_residues = []
    seq_ids = []
    seed_proteins = []
    urls = []
    
    try:
        # Get the protein query string
        query_protein_string = table_driver.find_element_by_id('query-id').get_attribute('data-querychain').replace('_', '')
    except:
        return names, descriptions, rmsds, aligned_residues, seed_protein_total_residues, seq_ids, seed_proteins, urls
    
    # Get the protein length of queried sequence
    rulers = table_driver.find_elements_by_class_name('svg-mol-ruler-text')
    query_protein_length = int(rulers[-1].get_attribute('innerHTML'))

    # Change filter options at top
    redundancy_select = Select(table_driver.find_element_by_id('subset-select'))
    display_select = Select(table_driver.find_element_by_id('display-selection'))
    redundancy_select.select_by_visible_text('Medium redundancy')
    display_select.select_by_visible_text('Table')
    
    # Refresh table with new options
    refresh_table_button = table_driver.find_element_by_id('seqlist-button')
    refresh_table_button.click()
    
    # For navigating whole length of table
    pages = []      
    try:
        num_pages_menu = WebDriverWait(table_driver, 10).until(
            EC.presence_of_element_located((By.ID, "page-select-1"))
        )
        
        num_pages_menu_sel = Select(num_pages_menu)
        for o in num_pages_menu_sel.options:
            pages.append(o.text)
    except:
        pass

    # Case 1: Only 1 page
    if not pages:
        # Extract the data from a page of VAST
        table = table_driver.find_element_by_id('vast-table')
        table_html = table.get_attribute('outerHTML')
        table_bsoup = BeautifulSoup(table_html, 'html.parser')
        rows = table_bsoup.find_all('tr')
        rows.pop(0) # First row is always the table header
        
        for r in rows:
            row_values = r.text.strip().split('\n')
            r_url = 'https://www.ncbi.nlm.nih.gov/Structure/vast/' + r.find_all('a')[1]['href']
            
            # Filter conditions 
            ## 1. Only 1 sequence of PDB entry (no domains)
            ## 2. URL isn't in known or unknown urls (we've seen or are going to see the page)
            ## 3. Sequence identity < 90%
            ## 4. Matched residue length is at least 50% of the length of query protein
            
            if (len(row_values[0]) == 5 and float(row_values[5]) < 90 and 
                    float(row_values[1]) > query_protein_length * aligned_threshold and
                    r_url not in known_urls and r_url not in unknown_urls):
                names.append(row_values[0])
                descriptions.append(row_values[6])
                rmsds.append(row_values[4])
                aligned_residues.append(row_values[1])
                seed_protein_total_residues.append(query_protein_length)
                seq_ids.append(row_values[5])
                seed_proteins.append(query_protein_string)

                # This is the link to the same table, queried for the matched sequence (not just PDB ID)
                urls.append(r_url)
    else:
        # Case 2:Have to iterate through all pages of the table
        for option in pages:
            num_pages_menu = WebDriverWait(table_driver, 10).until(
                EC.presence_of_element_located((By.ID, "page-select-1"))
            )
            num_pages_menu_sel = Select(num_pages_menu)
            num_pages_menu_sel.select_by_visible_text(option)
            
            # Extract the data from a page of VAST
            table = table_driver.find_element_by_id('vast-table')
            table_html = table.get_attribute('outerHTML')
            table_bsoup = BeautifulSoup(table_html, 'html.parser')
            rows = table_bsoup.find_all('tr')
            rows.pop(0) # First row is always the table header

            for r in rows:
                row_values = r.text.strip().split('\n')
                r_url = 'https://www.ncbi.nlm.nih.gov/Structure/vast/' + r.find_all('a')[1]['href']

                # Filter conditions 
                ## 1. Only 1 sequence of PDB entry (no domains)
                ## 2. URL isn't in known or unknown urls (we've seen or are going to see the page)
                ## 3. Sequence identity < 90%
                ## 4. Matched residue length is at least 50% of the length of query protein

                if (len(row_values[0]) == 5 and float(row_values[5]) < 90 and 
                        float(row_values[1]) > query_protein_length * aligned_threshold and
                        r_url not in known_urls and r_url not in unknown_urls):
                    names.append(row_values[0])
                    descriptions.append(row_values[6])
                    rmsds.append(row_values[4])
                    aligned_residues.append(row_values[1])
                    seed_protein_total_residues.append(query_protein_length)
                    seq_ids.append(row_values[5])
                    seed_proteins.append(query_protein_string)

                    # This is the link to the same table, queried for the matched sequence (not just PDB ID)
                    urls.append(r_url)
        
    return names, descriptions, rmsds, aligned_residues, seed_protein_total_residues, seq_ids, seed_proteins, urls

In [5]:
def get_all_matching_proteins(pdb_id: str, known_urls: list):
    unknown_urls = []
    
    names = []
    descriptions = []
    rmsds = []
    aligned_residues = [] 
    seed_protein_total_residues = []
    seq_ids = []
    seed_proteins = []
    
    rejected_protein_count = 0
    count = 0
    
    # initialize our unknown_urls list
    unknown_urls.append(get_seed_protein_url(pdb_id))
        
                
    while unknown_urls: # while unknown proteins is not empty
        current_url = unknown_urls.pop(0)
        
        print("Current url: " + current_url)
        
        if current_url not in known_urls:
            known_urls.append(current_url)
            
            threshold = 0.5 if (count == 0) else 0.75 # First round is more forgiving in protein alignment length
            names_1, descriptions_1, rmsds_1, aligned_residues_1, seed_protein_total_residues_1, seq_ids_1, seed_proteins_1, urls_1 = \
                get_info_from_url(current_url, known_urls, unknown_urls, threshold)
            
            names.extend(names_1)
            descriptions.extend(descriptions_1)
            rmsds.extend(rmsds_1)
            aligned_residues.extend(aligned_residues_1)
            seed_protein_total_residues.extend(seed_protein_total_residues_1)
            seq_ids.extend(seq_ids_1)
            seed_proteins.extend(seed_proteins_1)
            unknown_urls.extend(urls_1)
            
        else:
            rejected_protein_count += 1
            
        count += 1
            
    print("Queried protein count: " + str(count))
    protein_dict = {
        'name': names,
        'seed_protein': seed_proteins,
        'description': descriptions,
        'rmsd': rmsds,
        'aligned_residue': aligned_residues,
        'seed_protein_total_residues': seed_protein_total_residues,
        'sequence_id': seq_ids,
    }

    df = pd.DataFrame(protein_dict)
    return df, known_urls

In [ ]:
df = get_all_matching_proteins('1L1N')

In [ ]:
df 

In [ ]:
df.to_csv('./dataset_2/1L1N_med.csv')

## Now we need to repeat above process for multiple seed proteins!

In [11]:
pa_proteins = {
#    'C03': ['1L1NA', '1HAVA', '1CQQA', '2J92A', '2ZU3A', '2HRVA'],
#     'C04': ['1LVM'],
#     'C37': ['1WQSA'],
    'S01': #['2GMT', '1IAU', '1A0L', '2PSY', '2OQ5', '1OP0A', '2F91B', '1FI8A',
            #['1BRA', '1SGT', '1TRY', '2W5EA', '1DPO', 
            #['1HYL', '1AZZA', '1A0J',
            #['1TRN', '1HNE', '1CGH', '1FUJ', '1ORF', '1ORF', '3FZZ', 
            #['2ZGCA', '3RP2', '2F9P', '1MZA', '2RDLA', '1JRS', '2JETA', '1ELC', '1EKB', 
            #['1PYTC', '3DFJ', '2ZCHP', '1SGFG', '1GVZ', '1TON', '1AO5', 
            # [ '2R9PA', ????? A_0??
            #['1MD7A', '1ELV', '2ODP', '3GOVA', '2OLG', '1ZJDA', '1PFX', '1DAN',
            #'1HCG', '1ABJ', '1AUT', '1FIZ', '1O5FL', '1YBW', '1Q3X', '1MLW',
            #'1BUIA', '1LO6A', '1YM0', 
            ['1NPM', '2BDGA', '1SGC', '1SGPE', 
            '1HPG', '2ALP', '1QY6', '1EXF', '1KY9A', '1SO7', '1LCYA', '1ARC',
            '2VID', '2AS9A', '2QXIA', '2GV6A', '2SFA', '1EQ9', '1P3E', 
            '2AIQA', '1L1J', '2XXL', '1SGFA', '2PFE'
           ],
#     'S03': ['2SNV'],
#     'S06': ['1WXR'],
#     'S29': ['1A1R'],
#     'S32': ['1MBMA'],
#     'S39': ['1ZYO']
    
}

In [12]:
for family, pdb_list in pa_proteins.items():
    for p in pdb_list:
        known_proteins_file = open('dataset_2/known_proteins.txt', 'r+')
        known_proteins = known_proteins_file.readlines()
        
        df, new_known_proteins = get_all_matching_proteins(p, known_proteins)
        
        new_known_proteins.append(p)
        
        # Write out new data
        outdir = './dataset_2/' + family
        if not os.path.exists(outdir):
            os.mkdir(outdir)
            
        df.to_csv('./dataset_2/' + family + '/' + p + '.csv')
        known_proteins_file.writelines(s + '\n' for s in new_known_proteins)
        
        del new_known_proteins[:]
        del known_proteins
        del df
        

Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=27739
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=769899
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=21488
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=262531
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=562746
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=63207
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=45234
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=139131
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=98192
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=408632
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=496690
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=799326
Current url: https://www.ncbi.nlm

Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=581256
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=11841
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=801445
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=434812
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=631452
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=732052
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=652806
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=672025
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=665899
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=676509
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=641252
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=527886
Current url: https://www.ncbi

Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=337679
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=801430
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=801409
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=801424
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=21489
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=425867
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=676722
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=79571
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=155653
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=254402
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=28050
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=581256
Current url: https://www.ncbi.n

Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=722444
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=398561
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=660298
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=745301
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=802980
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=323038
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=802975
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=98192
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=786629
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=568305
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=778270
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=769899
Current url: https://www.ncbi

Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=746649
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=285887
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=786635
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=759731
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=802166
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=801409
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=807503
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=801430
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=801424
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=14427
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=614234
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=39383
Current url: https://www.ncbi.

...

Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=812083
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=6969
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=75258
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=224850
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=786629
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=799748
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=802166
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=337679
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=801448
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=746649
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=632322
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=448881
Current url: https://www.ncbi.n

Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=437182
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=802980
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=45234
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=177958
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=706228
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=660220
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=802975
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=568305
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=194702
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=139131
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=307612
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=807503
Current url: https://www.ncbi

Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=744212
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=111169
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=17701
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=319984
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=408667
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=45232
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=135925
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=107387
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=807931
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=237474
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=288506
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=448881
Current url: https://www.ncbi.

Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=386746
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=527876
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=122750
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=393535
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=807931
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=111169
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=278912
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=722444
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=325975
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=17701
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=237474
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=568305
Current url: https://www.ncbi

Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=285887
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=592295
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=37914
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=111169
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=191655
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=144889
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=325978
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=799326
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=17701
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=158452
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=746658
Current url: https://www.ncbi.nlm.nih.gov/Structure/vast/vastsrv.cgi?sdid=43608
Current url: https://www.ncbi.n